# Applied Data Science Capstone Report

## 1. Introduction/Businees problem

This study is based on helping people planning to open a new restaurant in Toronto to chose the right location by providing data about the  income and crime rate of each neighborhood for weighing safety and richness. There are many people in Toronto with high density. it is recommanded to make their restaurant properly

## 2.Data

The data is from foursquare.
To provide the stakeholders the necessary information I'll be combining Toronto's 2016 Census that contains Population, Average income per Neighborhood with Foursquare API to collect competitors on the same neighborhoods.

In [2]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

!pip install geopy # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

!pip install folium # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

     |████████████████████████████████| 102kB 7.8MB/s ta 0:00:011
Libraries imported.


### Bring income data for toronto(2016)

In [23]:
csv_path='https://www.toronto.ca/ext/open_data/catalog/data_set_files/2016_neighbourhood_profiles.csv'
df = pd.read_csv(csv_path,encoding='latin1')
print('Data loaded')

Data loaded


In [24]:
df.head()

,Category,Topic,Data Source,Characteristic,City of Toronto,Agincourt North,Agincourt South-Malvern West,Alderwood,Annex,Banbury-Don Mills,Bathurst Manor,Bay Street Corridor,Bayview Village,Bayview Woods-Steeles,Bedford Park-Nortown,Beechborough-Greenbrook,Bendale,Birchcliffe-Cliffside,Black Creek,Blake-Jones,Briar Hill-Belgravia,Bridle Path-Sunnybrook-York Mills,Broadview North,Brookhaven-Amesbury,Cabbagetown-South St. James Town,Caledonia-Fairbank,Casa Loma,Centennial Scarborough,Church-Yonge Corridor,Clairlea-Birchmount,Clanton Park,Cliffcrest,Corso Italia-Davenport,Danforth,Danforth East York,Don Valley Village,Dorset Park,Dovercourt-Wallace Emerson-Junction,Downsview-Roding-CFB,Dufferin Grove,East End-Danforth,Edenbridge-Humber Valley,Eglinton East,Elms-Old Rexdale,Englemount-Lawrence,Eringate-Centennial-West Deane,Etobicoke West Mall,Flemingdon Park,Forest Hill North,Forest Hill South,Glenfield-Jane Heights,Greenwood-Coxwell,Guildwood,Henry Farm,High Park North,High Park-Swansea,Highland Creek,Hillcrest Village,Humber Heights-Westmount,Humber Summit,Humbermede,Humewood-Cedarvale,Ionview,Islington-City Centre West,Junction Area,Keelesdale-Eglinton West,Kennedy Park,Kensington-Chinatown,Kingsview Village-The Westway,Kingsway South,Lambton Baby Point,L'Amoreaux,Lansing-Westgate,Lawrence Park North,Lawrence Park South,Leaside-Bennington,Little Portugal,Long Branch,Malvern,Maple Leaf,Markland Wood,Milliken,Mimico (includes Humber Bay Shores),Morningside,Moss Park,Mount Dennis,Mount Olive-Silverstone-Jamestown,Mount Pleasant East,Mount Pleasant West,New Toronto,Newtonbrook East,Newtonbrook West,Niagara,North Riverdale,North St. James Town,Oakridge,Oakwood Village,O'Connor-Parkview,Old East York,Palmerston-Little Italy,Parkwoods-Donalda,Pelmo Park-Humberlea,Playter Estates-Danforth,Pleasant View,Princess-Rosethorn,Regent Park,Rexdale-Kipling,Rockcliffe-Smythe,Roncesvalles,Rosedale-Moore Park,Rouge,Runnymede-Bloor West Village,Rustic,Scarborough Village,South Parkdale,South Riverdale,St.Andrew-Windfields,Steeles,Stonegate-Queensway,Tam O'Shanter-Sullivan,Taylor-Massey,The Beaches,Thistletown-Beaumond Heights,Thorncliffe Park,Trinity-Bellwoods,University,Victoria Village,Waterfront Communities-The Island,West Hill,West Humber-Clairville,Westminster-Branson,Weston,Weston-Pelham Park,Wexford/Maryvale,Willowdale East,Willowdale West,Willowridge-Martingrove-Richview,Woburn,Woodbine Corridor,Woodbine-Lumsden,Wychwood,Yonge-Eglinton,Yonge-St.Clair,York University Heights,Yorkdale-Glen Park
0,Neighbourhood Information,Neighbourhood Information,City of Toronto,Neighbourhood Number,NaN,129,128,20,95,42,34,76,52,49,39,112,127,122,24,69,108,41,57,30,71,109,96,133,75,120,33,123,92,66,59,47,126,93,26,83,62,9,138,5,32,11,13,44,102,101,25,65,140,53,88,87,134,48,8,21,22,106,125,14,90,110,124,78,6,15,114,117,38,105,103,56,84,19,132,29,12,130,17,135,73,115,2,99,104,18,50,36,82,68,74,121,107,54,58,80,45,23,67,46,10,72,4,111,86,98,131,89,28,139,85,70,40,116,16,118,61,63,3,55,81,79,43,77,136,1,35,113,91,119,51,37,7,137,64,60,94,100,97,27,31
1,Neighbourhood Information,Neighbourhood Information,City of Toronto,TSNS2020 Designation,NaN,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,NIA,No Designation,No Designation,NIA,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,Emerging Neighbourhood,No Designation,NIA,No Designation,No Designation,No Designation,NIA,NIA,Emerging Neighbourhood,No Designation,No Designation,NIA,No Designation,No Designation,NIA,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,Emerging Neighbourhood,NIA,NIA,No Designation,NIA,No Designation,No Designation,NIA,NIA,No Designat

In [72]:
csv_path='https://opendata.arcgis.com/datasets/af500b5abb7240399853b35a2362d0c0_0.csv?outSR=%7B%22latestWkid%22%3A26717%2C%22wkid%22%3A26717%7D'
df1 = pd.read_csv(csv_path,encoding='latin1')
print('Data loaded')


Data loaded


In [73]:

df1=df1.sort_values('Neighbourhood')
df1.head()


,OBJECTID,Neighbourhood,Hood_ID,Population,Assault_2014,Assault_2015,Assault_2016,Assault_2017,Assault_2018,Assault_2019,Assault_AVG,Assault_CHG,Assault_Rate_2019,AutoTheft_2014,AutoTheft_2015,AutoTheft_2016,AutoTheft_2017,AutoTheft_2018,AutoTheft_2019,AutoTheft_AVG,AutoTheft_CHG,AutoTheft_Rate_2019,BreakandEnter_2014,BreakandEnter_2015,BreakandEnter_2016,BreakandEnter_2017,BreakandEnter_2018,BreakandEnter_2019,BreakandEnter_AVG,BreakandEnter_CHG,BreakandEnter_Rate_2019,Homicide_2014,Homicide_2015,Homicide_2016,Homicide_2017,Homicide_2018,Homicide_2019,Homicide_AVG,Homicide_CHG,Homicide_Rate_2019,Robbery_2014,Robbery_2015,Robbery_2016,Robbery_2017,Robbery_2018,Robbery_2019,Robbery_AVG,Robbery_CHG,Robbery_Rate_2019,TheftOver_2014,TheftOver_2015,TheftOver_2016,TheftOver_2017,TheftOver_2018,TheftOver_2019,TheftOver_AVG,TheftOver_CHG,TheftOver_Rate_2019,Shape__Area,Shape__Length
79,80,Agincourt North,129,29113,67,76,77,71,79,79,74.8,0.00,271.4,17,29,16,32,42,42,29.7,0.00,144.3,46,52,49,49,69,56,53.5,-0.19,192.4,1,0,0,0,0,0,0.2,0.0,0.0,33,36,13,22,42,35,30.2,-0.17,120.2,1,5,5,11,4,2,4.7,-0.50,6.9,7.261857e+06,12381.264970
80,81,Agincourt South-Malvern West,128,23757,107,112,136,115,114,123,117.8,0.08,517.7,32,29,26,35,36,62,36.7,0.72,261.0,73,60,61,92,93,100,79.8,0.08,420.9,0,0,0,0,1,0,0.2,-1.0,0.0,24,23,21,34,33,29,27.3,-0.12,122.1,19,11,11,14,10,15,13.3,0.50,63.1,7.873163e+06,15386.556296
86,87,Alderwood,20,12054,45,47,39,21,30,36,36.3,0.20,298.7,9,18,17,14,25,14,16.2,-0.44,116.1,23,27,26,23,23,26,24.7,0.13,215.7,1,0,0,0,0,0,0.2,0.0,0.0,5,6,11,6,8,5,6.8,-0.38,41.5,7,3,10,7,7,7,6.8,0.00,58.1,4.978488e+06,9168.698727
56,57,Annex,95,30526,213,226,257,248,246,288,246.3,0.17,943.5,13,14,19,23,33,30,22.0,-0.09,98.3,156,106,123,116,172,212,147.5,0.23,694.5,0,0,0,0,2,1,0.5,-0.5,3.3,34,40,34,64,42,31,40.8,-0.26,101.6,22,17,22,29,45,42,29.5,-0.07,137.6,2.790356e+06,7606.039230
84,85,Banbury-Don Mills,42,27695,61,78,84,109,77,74,80.5,-0.04,267.2,19,20,7,16,27,42,21.8,0.56,151.7,65,87,57,64,85,81,73.2,-0.05,292.5,0,0,0,0,0,0,0.0,0.0,0.0,13,29,21,10,7,10,15.0,0.43,36.1,10,14,11,5,8,14,10.3,0.75,50.6,1.004155e+07,18165.123920


In [74]:
Neighbourhoods1 = list(df1.columns.values)

print(Neighbourhoods1)

['OBJECTID', 'Neighbourhood', 'Hood_ID', 'Population', 'Assault_2014', 'Assault_2015', 'Assault_2016', 'Assault_2017', 'Assault_2018', 'Assault_2019', 'Assault_AVG', 'Assault_CHG', 'Assault_Rate_2019', 'AutoTheft_2014', 'AutoTheft_2015', 'AutoTheft_2016', 'AutoTheft_2017', 'AutoTheft_2018', 'AutoTheft_2019', 'AutoTheft_AVG', 'AutoTheft_CHG', 'AutoTheft_Rate_2019', 'BreakandEnter_2014', 'BreakandEnter_2015', 'BreakandEnter_2016', 'BreakandEnter_2017', 'BreakandEnter_2018', 'BreakandEnter_2019', 'BreakandEnter_AVG', 'BreakandEnter_CHG', 'BreakandEnter_Rate_2019', 'Homicide_2014', 'Homicide_2015', 'Homicide_2016', 'Homicide_2017', 'Homicide_2018', 'Homicide_2019', 'Homicide_AVG', 'Homicide_CHG', 'Homicide_Rate_2019', 'Robbery_2014', 'Robbery_2015', 'Robbery_2016', 'Robbery_2017', 'Robbery_2018', 'Robbery_2019', 'Robbery_AVG', 'Robbery_CHG', 'Robbery_Rate_2019', 'TheftOver_2014', 'TheftOver_2015', 'TheftOver_2016', 'TheftOver_2017', 'TheftOver_2018', 'TheftOver_2019', 'TheftOver_AVG', 'The

In [75]:
Neighbourhoods = list(df.columns.values)
Neighbourhoods = Neighbourhoods[5:]
print(Neighbourhoods)


['Agincourt North', 'Agincourt South-Malvern West', 'Alderwood', 'Annex', 'Banbury-Don Mills', 'Bathurst Manor', 'Bay Street Corridor', 'Bayview Village', 'Bayview Woods-Steeles', 'Bedford Park-Nortown', 'Beechborough-Greenbrook', 'Bendale', 'Birchcliffe-Cliffside', 'Black Creek', 'Blake-Jones', 'Briar Hill-Belgravia', 'Bridle Path-Sunnybrook-York Mills', 'Broadview North', 'Brookhaven-Amesbury', 'Cabbagetown-South St. James Town', 'Caledonia-Fairbank', 'Casa Loma', 'Centennial Scarborough', 'Church-Yonge Corridor', 'Clairlea-Birchmount', 'Clanton Park', 'Cliffcrest', 'Corso Italia-Davenport', 'Danforth', 'Danforth East York', 'Don Valley Village', 'Dorset Park', 'Dovercourt-Wallace Emerson-Junction', 'Downsview-Roding-CFB', 'Dufferin Grove', 'East End-Danforth', 'Edenbridge-Humber Valley', 'Eglinton East', 'Elms-Old Rexdale', 'Englemount-Lawrence', 'Eringate-Centennial-West Deane', 'Etobicoke West Mall', 'Flemingdon Park', 'Forest Hill North', 'Forest Hill South', 'Glenfield-Jane Heig

In [94]:
dfT = pd.DataFrame(index=Neighbourhoods, columns=["Income_2016"])
dfT.head()

,Income_2016
Agincourt North,NaN
Agincourt South-Malvern West,NaN
Alderwood,NaN
Annex,NaN
Banbury-Don Mills,NaN


In [95]:
df1=df1.reset_index()
df1.head()

,level_0,index,OBJECTID,Neighbourhood,Hood_ID,Population,Assault_2014,Assault_2015,Assault_2016,Assault_2017,Assault_2018,Assault_2019,Assault_AVG,Assault_CHG,Assault_Rate_2019,AutoTheft_2014,AutoTheft_2015,AutoTheft_2016,AutoTheft_2017,AutoTheft_2018,AutoTheft_2019,AutoTheft_AVG,AutoTheft_CHG,AutoTheft_Rate_2019,BreakandEnter_2014,BreakandEnter_2015,BreakandEnter_2016,BreakandEnter_2017,BreakandEnter_2018,BreakandEnter_2019,BreakandEnter_AVG,BreakandEnter_CHG,BreakandEnter_Rate_2019,Homicide_2014,Homicide_2015,Homicide_2016,Homicide_2017,Homicide_2018,Homicide_2019,Homicide_AVG,Homicide_CHG,Homicide_Rate_2019,Robbery_2014,Robbery_2015,Robbery_2016,Robbery_2017,Robbery_2018,Robbery_2019,Robbery_AVG,Robbery_CHG,Robbery_Rate_2019,TheftOver_2014,TheftOver_2015,TheftOver_2016,TheftOver_2017,TheftOver_2018,TheftOver_2019,TheftOver_AVG,TheftOver_CHG,TheftOver_Rate_2019,Shape__Area,Shape__Length
0,0,79,80,Agincourt North,129,29113,67,76,77,71,79,79,74.8,0.00,271.4,17,29,16,32,42,42,29.7,0.00,144.3,46,52,49,49,69,56,53.5,-0.19,192.4,1,0,0,0,0,0,0.2,0.0,0.0,33,36,13,22,42,35,30.2,-0.17,120.2,1,5,5,11,4,2,4.7,-0.50,6.9,7.261857e+06,12381.264970
1,1,80,81,Agincourt South-Malvern West,128,23757,107,112,136,115,114,123,117.8,0.08,517.7,32,29,26,35,36,62,36.7,0.72,261.0,73,60,61,92,93,100,79.8,0.08,420.9,0,0,0,0,1,0,0.2,-1.0,0.0,24,23,21,34,33,29,27.3,-0.12,122.1,19,11,11,14,10,15,13.3,0.50,63.1,7.873163e+06,15386.556296
2,2,86,87,Alderwood,20,12054,45,47,39,21,30,36,36.3,0.20,298.7,9,18,17,14,25,14,16.2,-0.44,116.1,23,27,26,23,23,26,24.7,0.13,215.7,1,0,0,0,0,0,0.2,0.0,0.0,5,6,11,6,8,5,6.8,-0.38,41.5,7,3,10,7,7,7,6.8,0.00,58.1,4.978488e+06,9168.698727
3,3,56,57,Annex,95,30526,213,226,257,248,246,288,246.3,0.17,943.5,13,14,19,23,33,30,22.0,-0.09,98.3,156,106,123,116,172,212,147.5,0.23,694.5,0,0,0,0,2,1,0.5,-0.5,3.3,34,40,34,64,42,31,40.8,-0.26,101.6,22,17,22,29,45,42,29.5,-0.07,137.6,2.790356e+06,7606.039230
4,4,84,85,Banbury-Don Mills,42,27695,61,78,84,109,77,74,80.5,-0.04,267.2,19,20,7,16,27,42,21.8,0.56,151.7,65,87,57,64,85,81,73.2,-0.05,292.5,0,0,0,0,0,0,0.0,0.0,0.0,13,29,21,10,7,10,15.0,0.43,36.1,10,14,11,5,8,14,10.3,0.75,50.6,1.004155e+07,18165.123920


In [100]:
df1['Robbery_2016']

13

In [98]:
for index, row in dfT.iterrows():
    dfT.at[index, 'Income_2016'] = df[index][2264]
    
dfT

,Income_2016
Agincourt North,"30,414"
Agincourt South-Malvern West,"31,825"
Alderwood,"47,709"
Annex,"112,766"
Banbury-Don Mills,"67,757"
Bathurst Manor,"45,936"
Bay Street Corridor,"56,526"
Bayview Village,"52,035"
Bayview Woods-Steeles,"47,246"
Bedford Park-Nortown,"123,077"


In [105]:
dfT['Robbery_2016'] = dfT[['Income_2016']].apply(lambda x: ', '.join(x), axis=1 )
dfT.head()

,Income_2016,Robbery_2016
Agincourt North,"30,414","30,414"
Agincourt South-Malvern West,"31,825","31,825"
Alderwood,"47,709","47,709"
Annex,"112,766","112,766"
Banbury-Don Mills,"67,757","67,757"


In [117]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 140 entries, 0 to 139
Data columns (total 62 columns):
level_0                    140 non-null int64
index                      140 non-null int64
OBJECTID                   140 non-null int64
Neighbourhood              140 non-null object
Hood_ID                    140 non-null int64
Population                 140 non-null int64
Assault_2014               140 non-null int64
Assault_2015               140 non-null int64
Assault_2016               140 non-null int64
Assault_2017               140 non-null int64
Assault_2018               140 non-null int64
Assault_2019               140 non-null int64
Assault_AVG                140 non-null float64
Assault_CHG                140 non-null float64
Assault_Rate_2019          140 non-null float64
AutoTheft_2014             140 non-null int64
AutoTheft_2015             140 non-null int64
AutoTheft_2016             140 non-null int64
AutoTheft_2017             140 non-null int64
AutoTheft_2018

In [119]:
a=0
while a<140:
    dfT['Robbery_2016'][a]=str(df1['Robbery_2016'][a])
    a+=1

In [120]:
dfT.head()

,Income_2016,Robbery_2016
Agincourt North,"30,414",13
Agincourt South-Malvern West,"31,825",21
Alderwood,"47,709",11
Annex,"112,766",34
Banbury-Don Mills,"67,757",21


In [122]:
dfT.sort_values('Income_2016', ascend=True)

TypeError: sort_values() got an unexpected keyword argument 'ascend'